# Hyperwave Local Workflow

This notebook demonstrates the **local workflow** for FDTD photonics simulations.
Structure creation, density filtering, source generation, absorber setup, and analysis
all run **locally on your machine** using the hyperwave_community package.

Only the GPU simulation step (Step 7) runs on the cloud and requires an API key.
All other steps are completely free and require no authentication.

**For the API workflow** (simpler, high-level functions that handle structure creation on cloud CPUs),
see the [API Workflow tutorial](https://hyperwave-community.readthedocs.io/en/latest/workflows/api_workflow.html).

## Overview

**Device:** 2x2 MMI splitter (multimode interference coupler)

**Method:** 3D FDTD, local structure creation + cloud GPU simulation

**Time:** 10-15 minutes (including installation and local setup)

**Cost:** ~0.10 credits (single GPU simulation)

All structure creation, source generation, and analysis run locally on your machine.
Only the GPU simulation step runs on the cloud and requires an API key.

For on-premises GPU deployment, [contact our team](https://spinsphotonics.com/contact).

## Installation

In [ ]:
# Install hyperwave-community with gdsfactory
%pip install "hyperwave-community[gds] @ git+https://github.com/spinsphotonics/hyperwave-community.git" --no-cache-dir -q

In [ ]:
import hyperwave_community as hwc
import gdsfactory as gf
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp

# Activate generic PDK (required for gdsfactory)
PDK = gf.gpdk.get_generic_pdk()
PDK.activate()

---
## Step 1: Load GDSFactory Component

Load a photonic component from GDSFactory and convert it to a binary theta pattern.

In [ ]:
# Component settings
COMPONENT_NAME = "mmi2x2_with_sbend"
RESOLUTION_UM = 0.02  # 20nm resolution
EXTENSION_LENGTH = 2.0  # Extend ports by 2um (matches API workflow)

# Load component from GDSFactory (keep original for port-based monitor placement)
gf_device = gf.components.mmi2x2_with_sbend()

# Extend ports to ensure proper mode coupling
gf_extended = gf.c.extend_ports(gf_device, length=EXTENSION_LENGTH)

# Convert to theta pattern
theta, device_info = hwc.component_to_theta(
    component=gf_extended,
    resolution=RESOLUTION_UM,
)

print(f"Theta shape: {theta.shape}")
print(f"Device size: {device_info['physical_size_um']} um")

# Visualize
plt.figure(figsize=(12, 4))
plt.imshow(theta.T, cmap='gray', aspect='equal')
plt.title(f'{COMPONENT_NAME} - Theta Pattern')
plt.xlabel('x (pixels)')
plt.ylabel('y (pixels)')
plt.colorbar(label='Material (0=clad, 1=core)')
plt.show()

## Step 2: Apply Density Filtering

Smooth the theta pattern with density filtering. This helps with numerical stability.

In [ ]:
# Material properties
N_CORE = 3.48      # Silicon refractive index at 1550nm
N_CLAD = 1.45      # SiO2 cladding refractive index at 1550nm

# Padding for absorbers and monitors
PADDING = (100, 100, 0, 0)  # (left, right, top, bottom) in pixels

# Apply density filtering
density_core = hwc.density(
    theta=theta,
    pad_width=PADDING,
    radius=3,  # Smoothing radius for waveguide
)

density_clad = hwc.density(
    theta=jnp.zeros_like(theta),
    pad_width=PADDING,
    radius=5,  # Wider smoothing for uniform cladding
)

print(f"Density shape (with padding): {density_core.shape}")

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
ax1.imshow(density_core.T, cmap='PuOr')
ax1.set_title('Density Core (waveguide)')
ax2.imshow(density_clad.T, cmap='PuOr')
ax2.set_title('Density Clad (cladding only)')
plt.tight_layout()
plt.show()

## Step 3: Build 3D Layer Structure

Stack layers to create the 3D permittivity structure.

In [ ]:
# Layer dimensions
WG_HEIGHT_UM = 0.22         # Waveguide height (SOI standard)
TOTAL_HEIGHT_UM = 4.0       # Total simulation height
VERTICAL_RADIUS = 2         # Vertical blur radius

# Calculate layer thicknesses in cells
wg_thickness_cells = max(1, int(np.round(WG_HEIGHT_UM / RESOLUTION_UM)))
clad_thickness_cells = int(np.round((TOTAL_HEIGHT_UM - WG_HEIGHT_UM) / 2 / RESOLUTION_UM))

eps_core = N_CORE ** 2
eps_clad = N_CLAD ** 2

print(f"Waveguide: {wg_thickness_cells} cells ({WG_HEIGHT_UM*1000:.0f} nm)")
print(f"Cladding: {clad_thickness_cells} cells each")
print(f"Total Z: {clad_thickness_cells + wg_thickness_cells + clad_thickness_cells} cells")

# Define layers: bottom cladding / waveguide core / top cladding
waveguide_layer = hwc.Layer(
    density_pattern=density_core,
    permittivity_values=(eps_clad, eps_core),  # (clad, core)
    layer_thickness=wg_thickness_cells,
)

cladding_layer = hwc.Layer(
    density_pattern=density_clad,
    permittivity_values=eps_clad,
    layer_thickness=clad_thickness_cells,
)

# Create 3D structure
structure = hwc.create_structure(
    layers=[cladding_layer, waveguide_layer, cladding_layer],
    vertical_radius=VERTICAL_RADIUS,
)

_, Lx, Ly, Lz = structure.permittivity.shape
print(f"Structure dimensions: ({Lx}, {Ly}, {Lz})")

# Visualize cross-sections
z_mid = clad_thickness_cells + wg_thickness_cells // 2
structure.view(
    show_permittivity=True,
    show_conductivity=False,
    axis="z",
    position=z_mid,
    cmap_permittivity="PuOr",
)

structure.view(
    show_permittivity=True,
    show_conductivity=False,
    axis="x",
    position=Lx // 2,
    cmap_permittivity="PuOr",
)

## Step 4: Add Absorbing Boundaries

Add PML absorbing boundaries to prevent reflections.

In [ ]:
# Get Bayesian-optimized absorber parameters scaled for our resolution
abs_params = hwc.get_optimized_absorber_params(
    resolution_nm=RESOLUTION_UM * 1000,
    structure_dimensions=(Lx, Ly, Lz),
)

ABS_COEFF = abs_params['absorber_coeff']
abs_shape = abs_params['absorption_widths']

print(f"Absorber width (base): {abs_params['absorber_width']} cells")
print(f"Absorber coeff: {ABS_COEFF:.6f}")
print(f"Absorption widths (x,y,z): {abs_shape}")

# Create absorption mask
absorber = hwc.create_absorption_mask(
    grid_shape=(Lx, Ly, Lz),
    absorption_widths=abs_shape,
    absorption_coeff=ABS_COEFF,
    show_plots=True,
)

# Add absorber to structure conductivity
structure.conductivity = jnp.zeros_like(structure.conductivity) + absorber

# Visualize conductivity (absorber regions)
structure.view(
    show_permittivity=False,
    show_conductivity=True,
    axis="z",
    position=Lz // 2,
)

## Step 5: Create Mode Source

Solve for the fundamental waveguide mode at the input port.

In [ ]:
# Wavelength and frequency settings
WL_UM = 1.55  # Wavelength in microns
wl_cells = WL_UM / RESOLUTION_UM
freq_band = (2 * jnp.pi / wl_cells, 2 * jnp.pi / wl_cells, 1)

# Source position (after absorber region)
source_pos_x = abs_shape[0]

print(f"Wavelength: {WL_UM} um ({wl_cells:.0f} cells)")
print(f"Source position: x={source_pos_x}")

# Auto-detect waveguide bounds using temporary monitor placement
temp_monitors = hwc.MonitorSet()
temp_monitors.add_monitors_at_position(
    structure=structure,
    axis="x",
    position=source_pos_x,
    label="source_detect",
)

# Use the first detected waveguide (bottom input port)
source_monitor = temp_monitors.monitors[0]
print(f"Detected waveguide: shape={source_monitor.shape}, offset={source_monitor.offset}")

# Expand bounds to 2x for mode solving (ensures mode field decays to zero at edges)
y_min_orig = source_monitor.offset[1]
y_max_orig = y_min_orig + source_monitor.shape[1]
z_min_orig = source_monitor.offset[2]
z_max_orig = z_min_orig + source_monitor.shape[2]

y_center = (y_min_orig + y_max_orig) // 2
z_center = (z_min_orig + z_max_orig) // 2
y_half = source_monitor.shape[1]  # 2x original half-width
z_half = source_monitor.shape[2]  # 2x original half-height

y_min = max(0, y_center - y_half)
y_max = min(Ly, y_center + y_half)
z_min = max(0, z_center - z_half)
z_max = min(Lz, z_center + z_half)

print(f"Expanded mode region: Y=[{y_min}:{y_max}] ({y_max-y_min}px), Z=[{z_min}:{z_max}] ({z_max-z_min}px)")

# Create mode source in expanded region
source_field, source_offset, mode_info = hwc.create_mode_source(
    structure=structure,
    freq_band=freq_band,
    mode_num=0,  # Fundamental mode
    propagation_axis="x",
    source_position=source_pos_x,
    perpendicular_bounds=(y_min, y_max),
    z_bounds=(z_min, z_max),
    visualize=True,
    visualize_permittivity=True,
)

# Trim source field to actual mode region (create_mode_source pads to full structure)
# This reduces data transfer for the API simulation
source_field_trimmed = source_field[:, :, :, y_min:y_max, z_min:z_max]
source_offset_corrected = (source_pos_x, y_min, z_min)

print(f"\nOriginal source field: {source_field.shape}")
print(f"Trimmed source field: {source_field_trimmed.shape}")
print(f"Corrected offset: {source_offset_corrected}")

## Step 6: Set Up Monitors

Configure field monitors at input and output ports.

In [ ]:
# Create monitor set
monitors = hwc.MonitorSet()

# Coordinate mapping from device_info
y_pad_struct = PADDING[0] // 2  # Structure padding (density halves the padding)
x_pad_struct = PADDING[2] // 2
bbox = device_info['bounding_box_um']
x_min_um, y_min_um = bbox[0], bbox[1]
theta_res = device_info['theta_resolution_um']

# Monitor sizing
MONITOR_THICKNESS = 5
MONITOR_HALF = 35  # Half-extent in Y and Z (~1.4 um at 20nm resolution)
z_wg_center = clad_thickness_cells + wg_thickness_cells // 2

# Place monitors at original device ports (before port extension)
print("=== Monitors at device ports ===")
for port in gf_device.ports:
    px_um, py_um = port.center
    x_struct = int((px_um - x_min_um) / theta_res / 2) + x_pad_struct
    y_struct = int((py_um - y_min_um) / theta_res / 2) + y_pad_struct

    # Input ports face left (180 deg), output ports face right (0 deg)
    if abs(port.orientation % 360 - 180) < 1:
        label = f"Input_{port.name}"
    else:
        label = f"Output_{port.name}"

    monitor = hwc.Monitor(
        shape=(MONITOR_THICKNESS, 2 * MONITOR_HALF, 2 * MONITOR_HALF),
        offset=(x_struct, y_struct - MONITOR_HALF, z_wg_center - MONITOR_HALF)
    )
    monitors.add(monitor, label)
    print(f"  {label}: ({px_um:.2f}, {py_um:.2f}) um -> struct({x_struct}, {y_struct})")

# Add xy_mid monitor for field visualization (full XY plane at middle Z)
xy_mid_monitor = hwc.Monitor(
    shape=(Lx, Ly, 1),
    offset=(0, 0, z_wg_center)
)
monitors.add(xy_mid_monitor, name="xy_mid")

# List all monitors
monitors.list_monitors()

# Visualize monitor positions
monitors.view(
    structure=structure,
    axis="z",
    position=z_wg_center,
    source_position=source_pos_x,
    absorber_boundary=absorber,
)

---
## Step 7: Run GPU Simulation

This is the only step that runs on the cloud and requires an API key. The FDTD simulation
needs GPU memory and compute that would be impractical on most local machines. All other
steps in this notebook ran locally at no cost.

### API Key Setup

This notebook uses Colab Secrets to securely store your API key.
To set up your key:

1. Click the key icon in the left sidebar of this notebook
2. Add a new secret named `HYPERWAVE_API_KEY`
3. Paste your API key as the value
4. Toggle "Notebook access" to ON

If you don't have an API key, [sign up](https://spinsphotonics.com/signup) to get one for free.

In [ ]:
from google.colab import userdata
hwc.configure_api(api_key=userdata.get('HYPERWAVE_API_KEY'))
hwc.get_account_info();

In [ ]:
# Estimate cost before running
cost = hwc.estimate_cost(
    grid_points=Lx * Ly * Lz,
    max_steps=20000,
)
print(f"Estimated time: {cost['estimated_seconds']:.0f}s")
print(f"Estimated cost: ${cost['estimated_cost_usd']:.2f}")

In [ ]:
# Extract recipes for API
structure_recipe = structure.extract_recipe()
monitors_recipe = monitors.recipe

# Run FDTD simulation on cloud GPU
results = hwc.simulate(
    structure_recipe=structure_recipe,
    source_field=source_field_trimmed,
    source_offset=source_offset_corrected,
    freq_band=freq_band,
    monitors_recipe=monitors_recipe,
    mode_info=mode_info,
    simulation_steps=20000,
    check_every_n=1000,
    source_ramp_periods=5.0,
    add_absorption=True,
    absorption_widths=abs_shape,
    absorption_coeff=ABS_COEFF,
)

print(f"GPU time: {results['sim_time']:.2f}s")
print(f"Performance: {results['performance']:.2e} grid-points*steps/s")

In [ ]:
# Quick visualization of all monitors
hwc.quick_view_monitors(results, component="all")

---
## Step 8: Result Analysis

Analyze transmission and visualize fields locally.

In [ ]:
# Get monitor data
monitor_data = results['monitor_data']
print(f"Available monitors: {list(monitor_data.keys())}")

# MMI 2x2 port mapping (gdsfactory port names):
#   Input_o1 (bottom) ---> Output_o4 (bottom) [bar]
#   Input_o2 (top)    ---> Output_o3 (top)    [bar]
#
# Excited Input_o1 (bottom), so:
#   Bar port   = Output_o4 (same side)
#   Cross port = Output_o3 (opposite side)

input_name = "Input_o1"
bar_name = "Output_o4"     # Same side as input
cross_name = "Output_o3"   # Opposite side

input_fields = monitor_data[input_name]
bar_fields = monitor_data[bar_name]
cross_fields = monitor_data[cross_name]

print(f"Excited: {input_name}")
print(f"Monitor field shape: {input_fields.shape}")

# Average across monitor thickness (X dimension)
input_plane = jnp.mean(input_fields, axis=2)
bar_plane = jnp.mean(bar_fields, axis=2)
cross_plane = jnp.mean(cross_fields, axis=2)

# Calculate Poynting vectors
S_in = hwc.S_from_slice(input_plane)
S_bar = hwc.S_from_slice(bar_plane)
S_cross = hwc.S_from_slice(cross_plane)

# Calculate power (X-component for x-propagating mode)
power_in = jnp.abs(jnp.sum(S_in[:, 0, :, :], axis=(1, 2)))
power_bar = jnp.abs(jnp.sum(S_bar[:, 0, :, :], axis=(1, 2)))
power_cross = jnp.abs(jnp.sum(S_cross[:, 0, :, :], axis=(1, 2)))
power_out_total = power_bar + power_cross

# Calculate transmission metrics
T_bar = power_bar / power_in
T_cross = power_cross / power_in
T_total = power_out_total / power_in
excess_loss_dB = 10 * jnp.log10(T_total)

# Print results
print("\n" + "=" * 60)
print("MMI 2x2 POWER ANALYSIS")
print("=" * 60)
print(f"Bar port ({bar_name}):    T = {float(T_bar[0]):.5f}")
print(f"Cross port ({cross_name}): T = {float(T_cross[0]):.5f}")
print(f"Total transmission:        {float(T_total[0]):.5f}")
print(f"Excess loss:               {float(excess_loss_dB[0]):.2f} dB")
print(f"Split ratio: {float(power_bar[0]/power_out_total[0]*100):.1f}% / {float(power_cross[0]/power_out_total[0]*100):.1f}%")
print("=" * 60)


In [ ]:
# === FIELD INTENSITY VISUALIZATION ===
# Plot |E|^2 from xy_mid monitor

xy_mid_data = monitor_data['xy_mid']  # Shape: (N_freq, 6, Lx, Ly, 1)
print(f"xy_mid shape: {xy_mid_data.shape}")

# Calculate |E|^2 = |Ex|^2 + |Ey|^2 + |Ez|^2
E_fields = xy_mid_data[0, 0:3, :, :, 0]  # First freq, E components, squeeze Z
E_intensity = jnp.sum(jnp.abs(E_fields)**2, axis=0)

# Plot field intensity (pixel coordinates)
plt.figure(figsize=(14, 5))
plt.imshow(
    E_intensity.T,
    origin='lower',
    cmap='jet',
    aspect='equal'
)
plt.xlabel('x (pixels)')
plt.ylabel('y (pixels)')
plt.title("|E|^2 Field Intensity")
plt.colorbar(label='|E|^2')
plt.tight_layout()
plt.show()


---
## Summary

| Step | Function | Runs On | Cost |
|------|----------|---------|------|
| 1 | `component_to_theta()` | Local | Free |
| 2 | `density()` | Local | Free |
| 3 | `Layer()`, `create_structure()` | Local | Free |
| 4 | `get_optimized_absorber_params()`, `create_absorption_mask()` | Local | Free |
| 5 | `create_mode_source()` | Local | Free |
| 6 | `MonitorSet()`, `Monitor()` (port-based) | Local | Free |
| 7 | `simulate()` | Cloud GPU | Credits |
| 8 | `S_from_slice()`, analysis | Local | Free |

### Key Patterns

- **Density filtering**: Use `radius=3` for waveguide core, `radius=5` for uniform cladding
- **Absorber params**: Use `get_optimized_absorber_params()` for resolution-scaled values
- **Mode source**: Auto-detect waveguide bounds, expand 2x, trim source field after solving
- **Monitors**: Place at GDSFactory port positions for correct naming (Input/Output per port)
- **Analysis**: Use `hwc.S_from_slice()` for Poynting vector calculation